# Notebook 09: Model Training - Support Vector Machine (SVM)

**Objective:** Train and evaluate SVM classifier for crop recommendation

**Contents:**
1. Import Libraries and Load Data
2. SVM Algorithm Overview
3. Train Multiple SVM Models (Linear, RBF, Poly kernels)
4. Hyperparameter Tuning with GridSearchCV
5. Model Evaluation and Performance Metrics
6. Confusion Matrix Visualization
7. Support Vectors Analysis
8. Model Comparison and Selection
9. Save Best Model

# 1. Import Required Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                             f1_score, classification_report, confusion_matrix,
                             roc_curve, auc, roc_auc_score)
from sklearn.preprocessing import label_binarize
import pickle
import time
import warnings
warnings.filterwarnings('ignore')

In [ ]:
print("="*80)
print("NOTEBOOK 09: MODEL TRAINING - SUPPORT VECTOR MACHINE (SVM)")
print("="*80)

In [ ]:
# Set plotting style
plt.style.use("seaborn-v0_8-darkgrid")
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

# 2. Load Prepared Datasets

In [ ]:
# Load training and testing sets (Standard scaled)
X_train = np.load("../data/processed/ml_ready/X_train_scaled.npy")
X_test = np.load("../data/processed/ml_ready/X_test_scaled.npy")
y_train = np.load("../data/processed/ml_ready/y_train.npy")
y_test = np.load("../data/processed/ml_ready/y_test.npy")

In [ ]:
# Load feature names
with open("../data/processed/ml_ready/feature_names.pkl", "rb") as f:
    feature_names = pickle.load(f)

In [ ]:
# Load label encoder
with open("../models/label_encoder.pkl", "rb") as f:
    label_encoder = pickle.load(f)

In [ ]:
print("\n📊 DATASET SUMMARY:")
print("-" * 40)
print(f"Training samples: {X_train.shape[0]:,}")
print(f"Testing samples: {X_test.shape[0]:,}")
print(f"Number of features: {X_train.shape[1]:,}")
print(f"Number of classes: {len(np.unique(y_train)):,}")
print(f"\nClass labels: {label_encoder.classes_.tolist()}")

# 3. Support Vector Machine - Algorithm Overview

In [ ]:
overview = """
📚 SUPPORT VECTOR MACHINE (SVM):

🎯 TYPE: Maximum Margin Classification Algorithm

📐 MATHEMATICAL CONCEPT:
   • Finds the optimal hyperplane that maximizes the margin between classes
   • Decision boundary: w·x + b = 0
   • Support vectors are the closest points to the decision boundary
   
   Optimization objective:
   minimize: (1/2)||w||² + C × Σξᵢ
   subject to: yᵢ(w·xᵢ + b) ≥ 1 - ξᵢ
   
   where:
   • w = Weight vector (hyperplane normal)
   • b = Bias term
   • C = Regularization parameter
   • ξᵢ = Slack variables for soft margin

🔄 KERNEL FUNCTIONS:
   • Linear: K(x,y) = x·y
   • RBF (Gaussian): K(x,y) = exp(-γ||x-y||²)
   • Polynomial: K(x,y) = (γx·y + r)^d
   • Sigmoid: K(x,y) = tanh(γx·y + r)

🔄 FOR MULTI-CLASS (22 crops):
   • Uses One-vs-One (OvO) or One-vs-Rest (OvR) strategy
   • OvO: Creates n(n-1)/2 = 231 binary classifiers
   • OvR: Creates n = 22 binary classifiers

✅ ADVANTAGES:
   • Effective in high-dimensional spaces
   • Works well with non-linear boundaries (kernel trick)
   • Robust to outliers (margin maximization)
   • Memory efficient (only stores support vectors)
   • Excellent generalization capability

❌ LIMITATIONS:
   • Slow training on large datasets (O(n²) to O(n³))
   • Sensitive to feature scaling (requires normalization)
   • Choice of kernel and hyperparameters is crucial
   • Doesn't provide probability estimates directly

🎯 BEST FOR:
   • Medium-sized datasets
   • Binary and multi-class classification
   • High-dimensional feature spaces
   • When margin of separation is important
"""

print(overview)

# 4. Train Base SVM Models with Different Kernels

In [ ]:
print("\n" + "="*80)
print("TRAINING SVM MODELS WITH DIFFERENT KERNELS")
print("="*80)

# Define kernel configurations
kernel_configs = {
    'Linear': {'kernel': 'linear', 'C': 1.0},
    'RBF': {'kernel': 'rbf', 'C': 1.0, 'gamma': 'scale'},
    'Polynomial': {'kernel': 'poly', 'C': 1.0, 'degree': 3, 'gamma': 'scale'}
}

# Store results
svm_models = {}
kernel_results = []

In [ ]:
for kernel_name, params in kernel_configs.items():
    print(f"\n🔄 Training SVM with {kernel_name} kernel...")
    print(f"   Parameters: {params}")
    
    # Initialize and train model
    start_time = time.time()
    svm_model = SVC(**params, random_state=42, probability=True)
    svm_model.fit(X_train, y_train)
    training_time = time.time() - start_time
    
    # Make predictions
    start_time = time.time()
    y_train_pred = svm_model.predict(X_train)
    y_test_pred = svm_model.predict(X_test)
    prediction_time = time.time() - start_time
    
    # Calculate metrics
    train_accuracy = accuracy_score(y_train, y_train_pred)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    test_precision = precision_score(y_test, y_test_pred, average='weighted', zero_division=0)
    test_recall = recall_score(y_test, y_test_pred, average='weighted', zero_division=0)
    test_f1 = f1_score(y_test, y_test_pred, average='weighted', zero_division=0)
    
    # Store model and results
    svm_models[kernel_name] = svm_model
    kernel_results.append({
        'Kernel': kernel_name,
        'Train Accuracy': train_accuracy,
        'Test Accuracy': test_accuracy,
        'Precision': test_precision,
        'Recall': test_recall,
        'F1-Score': test_f1,
        'Training Time (s)': training_time,
        'Prediction Time (s)': prediction_time,
        'Support Vectors': svm_model.n_support_.sum(),
        'Overfitting Gap': train_accuracy - test_accuracy
    })
    
    print(f"   ✅ Completed in {training_time:.4f} seconds")
    print(f"   • Train Accuracy: {train_accuracy*100:.2f}%")
    print(f"   • Test Accuracy: {test_accuracy*100:.2f}%")
    print(f"   • Support Vectors: {svm_model.n_support_.sum()}")

In [ ]:
# Display kernel comparison
print("\n" + "="*80)
print("KERNEL COMPARISON RESULTS")
print("="*80)

results_df = pd.DataFrame(kernel_results)
print("\n")
print(results_df.to_string(index=False))

# 5. Hyperparameter Tuning with GridSearchCV

In [ ]:
print("\n" + "="*80)
print("HYPERPARAMETER TUNING WITH GRIDSEARCHCV")
print("="*80)

# Define parameter grid for RBF kernel (typically best performer)
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale', 'auto', 0.1, 0.01, 0.001],
    'kernel': ['rbf']
}

print("\n📋 Parameter Grid:")
for param, values in param_grid.items():
    print(f"   • {param}: {values}")

total_combinations = 1
for values in param_grid.values():
    total_combinations *= len(values)
print(f"\n   Total combinations: {total_combinations}")
print(f"   With 5-fold CV: {total_combinations * 5} fits")

In [ ]:
print("\n🔄 Running GridSearchCV (this may take a few minutes)...")

# Initialize GridSearchCV
svm_grid = GridSearchCV(
    estimator=SVC(random_state=42, probability=True),
    param_grid=param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1,
    return_train_score=True
)

# Fit GridSearchCV
start_time = time.time()
svm_grid.fit(X_train, y_train)
grid_search_time = time.time() - start_time

print(f"\n✅ GridSearchCV completed in {grid_search_time:.2f} seconds")

In [ ]:
print("\n" + "="*80)
print("BEST HYPERPARAMETERS")
print("="*80)

print(f"\n🏆 Best Parameters:")
for param, value in svm_grid.best_params_.items():
    print(f"   • {param}: {value}")

print(f"\n📊 Best Cross-Validation Score: {svm_grid.best_score_*100:.2f}%")

In [ ]:
# Display top 10 hyperparameter combinations
print("\n📋 Top 10 Hyperparameter Combinations:")

cv_results_df = pd.DataFrame(svm_grid.cv_results_)
cv_results_df = cv_results_df.sort_values('rank_test_score')

top_10 = cv_results_df[['params', 'mean_test_score', 'std_test_score', 'mean_train_score', 'rank_test_score']].head(10)
top_10['mean_test_score'] = top_10['mean_test_score'] * 100
top_10['mean_train_score'] = top_10['mean_train_score'] * 100
top_10['std_test_score'] = top_10['std_test_score'] * 100

print(top_10.to_string(index=False))

# 6. Train Final Model with Best Parameters

In [ ]:
print("\n" + "="*80)
print("TRAINING FINAL SVM MODEL WITH BEST PARAMETERS")
print("="*80)

# Get best model
best_svm = svm_grid.best_estimator_

# Make predictions
print("\n🔄 Making predictions...")
start_time = time.time()
y_train_pred = best_svm.predict(X_train)
y_train_pred_proba = best_svm.predict_proba(X_train)
train_pred_time = time.time() - start_time

start_time = time.time()
y_test_pred = best_svm.predict(X_test)
y_test_pred_proba = best_svm.predict_proba(X_test)
test_pred_time = time.time() - start_time

print(f"   Train prediction time: {train_pred_time:.4f} seconds")
print(f"   Test prediction time: {test_pred_time:.4f} seconds")
print(f"   Avg prediction time per sample: {test_pred_time/len(y_test)*1000:.4f} ms")

In [ ]:
# Show sample predictions
print("\n📋 Sample Predictions (First 10 test samples):")
sample_df = pd.DataFrame({
    "True Label": [label_encoder.classes_[i] for i in y_test[:10]],
    "Predicted": [label_encoder.classes_[i] for i in y_test_pred[:10]],
    "Confidence": [np.max(y_test_pred_proba[i]) * 100 for i in range(10)],
    "Match": ["✅" if y_test[i] == y_test_pred[i] else "❌" for i in range(10)]
})

print(sample_df.to_string(index=False))

# 7. Model Performance Metrics

In [ ]:
print("\n" + "="*80)
print("MODEL PERFORMANCE EVALUATION")
print("="*80)

# Training set metrics
train_accuracy = accuracy_score(y_train, y_train_pred)
train_precision = precision_score(y_train, y_train_pred, average='weighted', zero_division=0)
train_recall = recall_score(y_train, y_train_pred, average='weighted', zero_division=0)
train_f1 = f1_score(y_train, y_train_pred, average='weighted', zero_division=0)

# Testing set metrics
test_accuracy = accuracy_score(y_test, y_test_pred)
test_precision = precision_score(y_test, y_test_pred, average='weighted', zero_division=0)
test_recall = recall_score(y_test, y_test_pred, average='weighted', zero_division=0)
test_f1 = f1_score(y_test, y_test_pred, average='weighted', zero_division=0)

In [ ]:
print("\n📊 TRAINING SET PERFORMANCE:")
print("-" * 40)
print(f"   Accuracy:  {train_accuracy * 100:.2f}%")
print(f"   Precision: {train_precision * 100:.2f}%")
print(f"   Recall:    {train_recall * 100:.2f}%")
print(f"   F1-Score:  {train_f1 * 100:.2f}%")

In [ ]:
print("\n📊 TESTING SET PERFORMANCE:")
print("-" * 40)
print(f"   Accuracy:  {test_accuracy * 100:.2f}%")
print(f"   Precision: {test_precision * 100:.2f}%")
print(f"   Recall:    {test_recall * 100:.2f}%")
print(f"   F1-Score:  {test_f1 * 100:.2f}%")

In [ ]:
# Overfitting analysis
print("\n🔍 OVERFITTING ANALYSIS:")
print("-" * 40)
accuracy_diff = train_accuracy - test_accuracy
print(f"   Train-Test Accuracy Gap: {accuracy_diff * 100:.2f}%")

if accuracy_diff < 0.02:
    status = "✅ Excellent - No overfitting detected"
elif accuracy_diff < 0.05:
    status = "🟡 Good - Mild overfitting"
elif accuracy_diff < 0.10:
    status = "🟠 Moderate - Noticeable overfitting"
else:
    status = "🔴 Severe - Significant overfitting"

print(f"   Status: {status}")

# 8. Detailed Classification Report

In [ ]:
print("\n" + "="*80)
print("DETAILED CLASSIFICATION REPORT (Testing Set)")
print("="*80)

report = classification_report(y_test, y_test_pred, target_names=label_encoder.classes_, digits=4)
print(report)

In [ ]:
# Save classification report to CSV
report_dict = classification_report(y_test, y_test_pred, target_names=label_encoder.classes_, output_dict=True)
report_df = pd.DataFrame(report_dict).transpose()
report_df = report_df.round(4)
report_df.to_csv('../data/results/svm_classification_report.csv')
print("\n✅ Classification report saved to: ../data/results/svm_classification_report.csv")

# 9. Confusion Matrix Visualization

In [ ]:
print("\n" + "="*80)
print("CONFUSION MATRIX VISUALIZATION")
print("="*80)

# Calculate confusion matrix
cm = confusion_matrix(y_test, y_test_pred)

# Plot confusion matrix
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

# Raw counts
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=label_encoder.classes_, 
            yticklabels=label_encoder.classes_,
            ax=axes[0])
axes[0].set_title('SVM Confusion Matrix (Counts)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Predicted Label', fontsize=12)
axes[0].set_ylabel('True Label', fontsize=12)
axes[0].tick_params(axis='both', labelsize=8)
plt.setp(axes[0].xaxis.get_majorticklabels(), rotation=45, ha='right')

# Normalized
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
sns.heatmap(cm_normalized, annot=True, fmt='.2f', cmap='Blues',
            xticklabels=label_encoder.classes_,
            yticklabels=label_encoder.classes_,
            ax=axes[1])
axes[1].set_title('SVM Confusion Matrix (Normalized)', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Predicted Label', fontsize=12)
axes[1].set_ylabel('True Label', fontsize=12)
axes[1].tick_params(axis='both', labelsize=8)
plt.setp(axes[1].xaxis.get_majorticklabels(), rotation=45, ha='right')

plt.tight_layout()
plt.savefig('../data/visualizations/40_svm_confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✅ Confusion matrix saved to: ../data/visualizations/40_svm_confusion_matrix.png")

# 10. Support Vectors Analysis

In [ ]:
print("\n" + "="*80)
print("SUPPORT VECTORS ANALYSIS")
print("="*80)

print(f"\n📊 Total Support Vectors: {best_svm.n_support_.sum()}")
print(f"   Percentage of training data: {best_svm.n_support_.sum()/len(y_train)*100:.2f}%")

print("\n📋 Support Vectors per Class:")
sv_per_class = pd.DataFrame({
    'Class': label_encoder.classes_,
    'Support Vectors': best_svm.n_support_,
    'Percentage': (best_svm.n_support_ / best_svm.n_support_.sum() * 100).round(2)
})
print(sv_per_class.to_string(index=False))

In [ ]:
# Visualize support vectors per class
fig, ax = plt.subplots(figsize=(14, 6))

colors = plt.cm.husl(np.linspace(0, 1, len(label_encoder.classes_)))
bars = ax.bar(label_encoder.classes_, best_svm.n_support_, color=colors, edgecolor='black', linewidth=0.5)

ax.set_xlabel('Crop Class', fontsize=12)
ax.set_ylabel('Number of Support Vectors', fontsize=12)
ax.set_title('Support Vectors Distribution by Crop Class', fontsize=14, fontweight='bold')
plt.xticks(rotation=45, ha='right')

# Add value labels on bars
for bar, value in zip(bars, best_svm.n_support_):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5, 
            str(value), ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.savefig('../data/visualizations/41_svm_support_vectors.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✅ Support vectors plot saved to: ../data/visualizations/41_svm_support_vectors.png")

# 11. Per-Class Performance Analysis

In [ ]:
print("\n" + "="*80)
print("PER-CLASS PERFORMANCE ANALYSIS")
print("="*80)

# Create per-class performance visualization
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# Precision per class
precision_per_class = precision_score(y_test, y_test_pred, average=None, zero_division=0)
axes[0].barh(label_encoder.classes_, precision_per_class, color='steelblue', edgecolor='black')
axes[0].set_xlabel('Precision', fontsize=12)
axes[0].set_title('Precision by Crop', fontsize=14, fontweight='bold')
axes[0].axvline(x=np.mean(precision_per_class), color='red', linestyle='--', label=f'Mean: {np.mean(precision_per_class):.3f}')
axes[0].legend()

# Recall per class
recall_per_class = recall_score(y_test, y_test_pred, average=None, zero_division=0)
axes[1].barh(label_encoder.classes_, recall_per_class, color='forestgreen', edgecolor='black')
axes[1].set_xlabel('Recall', fontsize=12)
axes[1].set_title('Recall by Crop', fontsize=14, fontweight='bold')
axes[1].axvline(x=np.mean(recall_per_class), color='red', linestyle='--', label=f'Mean: {np.mean(recall_per_class):.3f}')
axes[1].legend()

# F1-Score per class
f1_per_class = f1_score(y_test, y_test_pred, average=None, zero_division=0)
axes[2].barh(label_encoder.classes_, f1_per_class, color='coral', edgecolor='black')
axes[2].set_xlabel('F1-Score', fontsize=12)
axes[2].set_title('F1-Score by Crop', fontsize=14, fontweight='bold')
axes[2].axvline(x=np.mean(f1_per_class), color='red', linestyle='--', label=f'Mean: {np.mean(f1_per_class):.3f}')
axes[2].legend()

plt.tight_layout()
plt.savefig('../data/visualizations/42_svm_per_class_performance.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✅ Per-class performance plot saved to: ../data/visualizations/42_svm_per_class_performance.png")

In [ ]:
# Identify best and difficult classes
print("\n🏆 BEST PERFORMING CLASSES (F1-Score):")
class_performance = pd.DataFrame({
    'Class': label_encoder.classes_,
    'Precision': precision_per_class,
    'Recall': recall_per_class,
    'F1-Score': f1_per_class
}).sort_values('F1-Score', ascending=False)

print("\nTop 5 classes:")
print(class_performance.head().to_string(index=False))

print("\n⚠️ CLASSES NEEDING IMPROVEMENT:")
print(class_performance.tail(5).to_string(index=False))

# 12. Prediction Confidence Analysis

In [ ]:
print("\n" + "="*80)
print("PREDICTION CONFIDENCE ANALYSIS")
print("="*80)

# Calculate confidence scores
confidence_scores = np.max(y_test_pred_proba, axis=1) * 100

# Confidence for correct vs incorrect predictions
correct_mask = y_test == y_test_pred
correct_confidence = confidence_scores[correct_mask]
incorrect_confidence = confidence_scores[~correct_mask]

print(f"\n📊 Confidence Statistics:")
print(f"   Overall mean confidence: {np.mean(confidence_scores):.2f}%")
print(f"   Correct predictions mean: {np.mean(correct_confidence):.2f}%")
if len(incorrect_confidence) > 0:
    print(f"   Incorrect predictions mean: {np.mean(incorrect_confidence):.2f}%")
print(f"   Min confidence: {np.min(confidence_scores):.2f}%")
print(f"   Max confidence: {np.max(confidence_scores):.2f}%")

In [ ]:
# Plot confidence distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
axes[0].hist(confidence_scores, bins=30, color='steelblue', edgecolor='black', alpha=0.7)
axes[0].axvline(x=np.mean(confidence_scores), color='red', linestyle='--', linewidth=2,
                label=f'Mean: {np.mean(confidence_scores):.1f}%')
axes[0].set_xlabel('Confidence Score (%)', fontsize=12)
axes[0].set_ylabel('Frequency', fontsize=12)
axes[0].set_title('SVM Prediction Confidence Distribution', fontsize=14, fontweight='bold')
axes[0].legend()

# Box plot comparing correct vs incorrect
if len(incorrect_confidence) > 0:
    data_to_plot = [correct_confidence, incorrect_confidence]
    bp = axes[1].boxplot(data_to_plot, labels=['Correct', 'Incorrect'], patch_artist=True)
    bp['boxes'][0].set_facecolor('lightgreen')
    bp['boxes'][1].set_facecolor('lightcoral')
else:
    bp = axes[1].boxplot([correct_confidence], labels=['Correct'], patch_artist=True)
    bp['boxes'][0].set_facecolor('lightgreen')
    
axes[1].set_ylabel('Confidence Score (%)', fontsize=12)
axes[1].set_title('Confidence: Correct vs Incorrect Predictions', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig('../data/visualizations/43_svm_confidence_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✅ Confidence distribution saved to: ../data/visualizations/43_svm_confidence_distribution.png")

# 13. Model Summary and Save

In [ ]:
print("\n" + "="*80)
print("SVM MODEL SUMMARY")
print("="*80)

summary = {
    'Model': 'Support Vector Machine',
    'Kernel': best_svm.kernel,
    'C (Regularization)': best_svm.C,
    'Gamma': best_svm.gamma if hasattr(best_svm, 'gamma') else 'N/A',
    'Train Accuracy': f'{train_accuracy*100:.2f}%',
    'Test Accuracy': f'{test_accuracy*100:.2f}%',
    'Test Precision': f'{test_precision*100:.2f}%',
    'Test Recall': f'{test_recall*100:.2f}%',
    'Test F1-Score': f'{test_f1*100:.2f}%',
    'Overfitting Gap': f'{accuracy_diff*100:.2f}%',
    'Total Support Vectors': best_svm.n_support_.sum(),
    'Training Time (GridSearch)': f'{grid_search_time:.2f}s'
}

print("\n📋 Model Configuration:")
for key, value in summary.items():
    print(f"   • {key}: {value}")

In [ ]:
# Save summary to CSV
summary_df = pd.DataFrame([{
    'Metric': 'Test Accuracy',
    'Value': f'{test_accuracy*100:.2f}%'
}, {
    'Metric': 'Test Precision',
    'Value': f'{test_precision*100:.2f}%'
}, {
    'Metric': 'Test Recall',
    'Value': f'{test_recall*100:.2f}%'
}, {
    'Metric': 'Test F1-Score',
    'Value': f'{test_f1*100:.2f}%'
}, {
    'Metric': 'Train Accuracy',
    'Value': f'{train_accuracy*100:.2f}%'
}, {
    'Metric': 'Overfitting Gap',
    'Value': f'{accuracy_diff*100:.2f}%'
}, {
    'Metric': 'Kernel',
    'Value': best_svm.kernel
}, {
    'Metric': 'C Parameter',
    'Value': str(best_svm.C)
}, {
    'Metric': 'Support Vectors',
    'Value': str(best_svm.n_support_.sum())
}])

summary_df.to_csv('../data/results/svm_summary.csv', index=False)
print("\n✅ Summary saved to: ../data/results/svm_summary.csv")

In [ ]:
# Save the model
print("\n💾 SAVING SVM MODEL...")

with open('../models/svm_model.pkl', 'wb') as f:
    pickle.dump(best_svm, f)

print("✅ Model saved to: ../models/svm_model.pkl")

# Verify saved model
import os
model_size = os.path.getsize('../models/svm_model.pkl') / 1024
print(f"   Model size: {model_size:.2f} KB")

In [ ]:
# Save predictions for comparison
predictions_df = pd.DataFrame({
    'True_Label_Encoded': y_test,
    'Predicted_Label_Encoded': y_test_pred,
    'True_Label': [label_encoder.classes_[i] for i in y_test],
    'Predicted_Label': [label_encoder.classes_[i] for i in y_test_pred],
    'Confidence': confidence_scores,
    'Correct': correct_mask
})

predictions_df.to_csv('../data/results/svm_predictions.csv', index=False)
print("✅ Predictions saved to: ../data/results/svm_predictions.csv")

# 14. Update Model Comparison

In [ ]:
print("\n" + "="*80)
print("UPDATING MODEL COMPARISON")
print("="*80)

# Load existing comparison if exists
try:
    comparison_df = pd.read_csv('../data/results/model_comparison_all.csv')
    print("\n📋 Existing models in comparison:")
    print(comparison_df['Metric'].tolist() if 'Metric' in comparison_df.columns else "Found comparison file")
except:
    comparison_df = None
    print("\n⚠️ No existing comparison file found. Creating new one.")

# Create new comparison including SVM
new_comparison = {
    'Metric': ['Test Accuracy', 'Test Precision', 'Test Recall', 'Test F1-Score', 
               'Training Time (sec)', 'Prediction Time (ms)', 'Overfitting Gap'],
    'SVM': [f'{test_accuracy*100:.2f}%', f'{test_precision*100:.2f}%', 
            f'{test_recall*100:.2f}%', f'{test_f1*100:.2f}%',
            f'{grid_search_time:.4f}', f'{test_pred_time/len(y_test)*1000:.4f}',
            f'{accuracy_diff*100:.2f}%']
}

if comparison_df is not None and 'Logistic_Regression' in comparison_df.columns:
    comparison_df['SVM'] = new_comparison['SVM']
else:
    comparison_df = pd.DataFrame(new_comparison)

comparison_df.to_csv('../data/results/model_comparison_all.csv', index=False)
print("\n✅ Model comparison updated: ../data/results/model_comparison_all.csv")
print("\n")
print(comparison_df.to_string(index=False))

# 15. Conclusion

In [ ]:
print("\n" + "="*80)
print("NOTEBOOK 09 COMPLETED SUCCESSFULLY!")
print("="*80)

print("""
📊 SVM MODEL TRAINING SUMMARY:

✅ Trained SVM models with 3 different kernels (Linear, RBF, Polynomial)
✅ Performed hyperparameter tuning using GridSearchCV with 5-fold CV
✅ Achieved {:.2f}% test accuracy with best model
✅ Generated comprehensive evaluation metrics and visualizations
✅ Analyzed support vectors distribution across classes
✅ Saved trained model and results

📁 OUTPUT FILES:
   • Model: ../models/svm_model.pkl
   • Classification Report: ../data/results/svm_classification_report.csv
   • Summary: ../data/results/svm_summary.csv
   • Predictions: ../data/results/svm_predictions.csv
   • Visualizations: ../data/visualizations/40-43_svm_*.png

🔄 NEXT STEPS:
   • Notebook 10: XGBoost and LightGBM Training
   • Notebook 11: Stacking Ensemble Model
   • Notebook 12: Final Model Comparison and Selection
""".format(test_accuracy*100))